In [1]:
import zipfile
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# # Function to load and preprocess images
# def load_and_preprocess_image(image_path):
#     img = image.load_img(image_path, target_size=(224, 224))
#     img_array = image.img_to_array(img)
#     return img_array/255.0




2024-06-29 02:18:06.779743: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-29 02:18:06.780746: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-29 02:18:06.827771: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-29 02:18:07.061832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 02:18:07.806375: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
from PIL import Image

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    try:
        img = Image.open(image_path)
        
        # Convert image to RGB if not already in RGB mode
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Resize image to target size
        img = img.resize(target_size)
        
        # Convert image to numpy array and normalize
        img_array = np.array(img) / 255.0
        
        return img_array
    
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [3]:
# Function to create data combinations
def create_data_combinations(image_folder):
    images, labels = [], []
    class_mapping = {
    'Audi': 0,
    'Rolls Royce': 1,
    'Toyota_celica': 2,
    'Toyota_hilux': 3,
    'Toyota_supra': 4,
    'BMW': 5,
    'Swift': 6,
    'Toyota_corolla': 7,
    'Toyota Innova': 8,
    'Toyota_yaris': 9,
    'Hyundai Creta': 10,
    'Tata Safari': 11,
    'Toyota_corona': 12,
    'Toyota_prius': 13,
    'Lamborghini': 14,
    'Toyota_alphard': 15,
    'Toyota_crown': 16,
    'Toyota_rav4': 17,
    'Mahindra Scorpio': 18,
    'Toyota_avanza': 19,
    'Toyota_fortuner': 20,
    'Toyota_rush': 21,
    'Mercedes': 22,
    'Toyota_camry': 23,
    'Toyota_hiace': 24

}




    for class_folder in os.listdir(image_folder):
        class_path = os.path.join(image_folder, class_folder)

        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img_data = load_and_preprocess_image(image_path)
            images.append(img_data)
            label = class_mapping[class_folder]
            labels.append(label)

        print(class_folder)

    return images, labels



In [ ]:
# Load dataset from local zipfile
local_zipfile_path = '/home/fahim/Music/car_model/zipfile.zip'
extract_folder = '/home/fahim/Music/CarsDataset'  # Change as per your folder structure

with zipfile.ZipFile(local_zipfile_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)



In [ ]:
image_folder = '/home/fahim/Music/CarsDataset/train/'
images,labels = create_data_combinations(image_folder)

Toyota_supra
Toyota_hilux
Audi
Toyota_rav4
Toyota_corona
Toyota_camry
Toyota_alphard
Rolls Royce


In [6]:
X_image = np.array(images)
Y_labels = np.array(labels)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X_image, Y_labels, test_size=0.2, random_state=42)

In [8]:
import numpy as np
from keras.models import load_model, Model
from keras.applications import VGG16
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Assuming you have already loaded and preprocessed your data into X_train, Y_train, X_test, Y_test
# Replace this with your actual data loading and preprocessing steps

# Example correction for labels in Y_train and Y_test
# Assuming labels need to be within range [0, 6]

# Check unique labels
unique_labels_train = np.unique(Y_train)
unique_labels_test = np.unique(Y_test)

print("Unique labels in Y_train:", unique_labels_train)
print("Unique labels in Y_test:", unique_labels_test)

# Correct labels if necessary
Y_train_corrected = np.clip(Y_train, 0, 6)  # Clip to ensure labels are within [0, 6]
Y_test_corrected = np.clip(Y_test, 0, 6)

# Replace Y_train and Y_test with corrected labels
Y_train = Y_train_corrected
Y_test = Y_test_corrected

# Load the pre-trained model
model_path = "/home/fahim/Music/car_model_classifier_vgg16_another.h5"  # Adjust path to your model

if os.path.exists(model_path):
    print("Loading pre-trained model from:", model_path)
    model = load_model(model_path)
else:
    print("No pre-trained model found. Training a new model.")
    # If model does not exist, train and save the model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(7, activation='softmax')(x)  # Adjust output units to 7 for your 7 classes

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test), callbacks=[early_stopping])

    # Save model
    model.save(model_path)


Unique labels in Y_train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
Unique labels in Y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
No pre-trained model found. Training a new model.



KeyboardInterrupt



In [25]:
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)
print('Y_train example:', Y_train[0])  # Print an example to verify its format


X_train shape: (864, 224, 224, 3)
Y_train shape: (864,)
X_test shape: (216, 224, 224, 3)
Y_test shape: (216,)
Y_train example: 2


In [ ]:
# # Prepare data
# image_folder = os.path.join(extract_folder, 'train')  # Adjust folder structure if needed
# images, labels = create_data_combinations(image_folder)

# X_image = np.array(images)
# Y_labels = np.array(labels)

# # Split data into train and test sets
# X_train, X_test, Y_train, Y_test = train_test_split(X_image, Y_labels, test_size=0.2, random_state=42)

# # Model definition and training
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# for layer in base_model.layers:
#     layer.trainable = False

# x = Flatten()(base_model.output)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# output = Dense(7, activation='softmax')(x)  # Adjust output units to 7 for your 7 classes

# model = Model(inputs=base_model.input, outputs=output)

# model.compile(optimizer=Adam(),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), callbacks=[early_stopping])


In [14]:

# # Save and use model for prediction
# model.save("/home/fahim/Music/car_model_classifier_vgg16_local.h5")  # Adjust save path as needed


In [26]:
# Function to preprocess locally uploaded image
def preprocess_local_image(image_path):
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))  # Resize image
        img = img.astype('float32') / 255.0
        return img
    except Exception as e:
        print("Error:", str(e))
        return None


In [27]:

# Example prediction function (similar to previous)
def preprocess_and_predict(image_path):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    return predicted_label

# Example usage
# uploaded_image_path = 'path_to_your_uploaded_image.jpg'
# predicted_label = preprocess_and_predict(uploaded_image_path)
# print("Predicted Label:", predicted_label)

In [28]:
# Function to preprocess locally uploaded image
def preprocess_local_image(image_path):
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))  # Resize image
        img = img.astype('float32') / 255.0
        return img
    except Exception as e:
        print("Error:", str(e))
        return None

# Function to predict car model from a local image file
def predict_car_model_local(image_path):
    preprocessed_image = preprocess_local_image(image_path)
    if preprocessed_image is None:
        return None
    
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
    prediction = model.predict(preprocessed_image)
    predicted_label_index = np.argmax(prediction)
    return predicted_label_index

class_mapping = {
    0: 'Audi',
    1: 'Rolls Royce',
    2: 'Toyota_celica',
    3: 'Toyota_hilux',
    4: 'Toyota_supra',
    5: 'BMW',
    6: 'Swift',
    7: 'Toyota_corolla',
    8: 'Toyota Innova',
    9: 'Toyota_yaris',
    10: 'Hyundai Creta',
    11: 'Tata Safari',
    12: 'Toyota_corona',
    13: 'Toyota_prius',
    14: 'Lamborghini',
    15: 'Toyota_alphard',
    16: 'Toyota_crown',
    17: 'Toyota_rav4',
    18: 'Mahindra Scorpio',
    19: 'Toyota_avanza',
    20: 'Toyota_fortuner',
    21: 'Toyota_rush',
    22: 'Mercedes',
    23: 'Toyota_camry',
    24: 'Toyota_hiace'
}



In [31]:
uploaded_image_path = '/home/fahim/Downloads/rr3.jpeg'  # Replace with your image path
predicted_car_model = predict_car_model_local(uploaded_image_path)

if predicted_car_model is not None:
    print("Predicted Car Model:", predicted_car_model)
else:
    print("Prediction failed. Please check your image path.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted Car Model: 1


In [ ]:
#for batch training

# Generator function for batches
def image_data_generator(image_folder, class_mapping, batch_size=32):
    images = []
    labels = []
    while True:
        for class_folder in os.listdir(image_folder):
            class_path = os.path.join(image_folder, class_folder)
            if not os.path.isdir(class_path):
                continue

            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                img_data = load_and_preprocess_image(image_path)
                if img_data is not None:
                    images.append(img_data)
                    label = class_mapping[class_folder]
                    labels.append(label)

                # Yield batch when the batch size is reached
                if len(images) == batch_size:
                    yield np.array(images), np.array(labels)
                    images, labels = [], []  # Reset for the next batch
        
        # Yield remaining images if any
        if images:
            yield np.array(images), np.array(labels)
            images, labels = [], []

# Class mapping dictionary
class_mapping = {
    'Audi': 0,
    'Rolls Royce': 1,
    'Toyota_celica': 2,
    'Toyota_hilux': 3,
    'Toyota_supra': 4,
    'BMW': 5,
    'Swift': 6,
    'Toyota_corolla': 7,
    'Toyota Innova': 8,
    'Toyota_yaris': 9,
    'Hyundai Creta': 10,
    'Tata Safari': 11,
    'Toyota_corona': 12,
    'Toyota_prius': 13,
    'Lamborghini': 14,
    'Toyota_alphard': 15,
    'Toyota_crown': 16,
    'Toyota_rav4': 17,
    'Mahindra Scorpio': 18,
    'Toyota_avanza': 19,
    'Toyota_fortuner': 20,
    'Toyota_rush': 21,
    'Mercedes': 22,
    'Toyota_camry': 23,
    'Toyota_hiace': 24
}

# Function to create data combinations
def create_data_combinations(image_folder):
    images, labels = [], []
    for class_folder in os.listdir(image_folder):
        class_path = os.path.join(image_folder, class_folder)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img_data = load_and_preprocess_image(image_path)
            if img_data is not None:
                images.append(img_data)
                label = class_mapping[class_folder]
                labels.append(label)

    return np.array(images), np.array(labels)




